In [1]:
import os
from dotenv import load_dotenv
from moviepy.editor import VideoFileClip, concatenate_videoclips, TextClip, ImageClip, CompositeVideoClip, AudioFileClip
import pyttsx3 as tts
import whisper
import requests
import google.generativeai as genai
load_dotenv()
from pyht import Client, TTSOptions, Format
import torch
torch.device("cuda" if torch.cuda.is_available() else "cpu")

client = Client("fn8dNXoXdOPpVq9srLrsoLtiLLL2", os.getenv('HT_API_KEY'))

gemini_key = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=gemini_key)

In [2]:
def generate_script(prompt, stories):
    model = genai.GenerativeModel('gemini-pro')
    default_prompt = """from now on return the output as regular text and the story should look
                    like it being narrated by someone. Write a Short and simple story,
                    You have to write maximum 170 words I want a plain text no symbols just the script, 
                    also the script content must be in 50 seconds duration, dont include any symbols or special characters,"""
    for i in range(stories):
        story = model.generate_content(f"{default_prompt},{prompt}")
        with open(f'stories/story_{i}.txt', 'w') as f:
            f.write(story.text)

In [3]:
def get_images(text):
    BASE = 'https://api.unsplash.com/photos/'
    PAYLOAD = {
        'query': text,
        'client_id': os.getenv('UNSPLASH_ACCESS_KEY')
    }
    response = requests.get(BASE, params=PAYLOAD)
    data = response.json()
    return data

In [4]:
prompt = "Scary Mystery Puzzle"
stories = 5
generate_script(prompt, stories)

In [5]:
# def eleven_speak(text_file, file_name):

#     voice_of = generate(text=text_file,
#                     voice = 'Adam')
#     play(voice_of)

def eleven_speak(text, file_name):
    engine = tts.init()
    engine.setProperty('rate', 200)
    engine.save_to_file(text, f'audio/{file_name}.mp3')
    engine.runAndWait()

for i in range(stories):
    with open(f'stories/story_{i}.txt', 'r') as f:
        text = f.readlines()
        text = [word.replace('\n', '') for word in text]
        new_text = ' '.join(text)
        print(new_text)
        eleven_speak(new_text, f'story_{i}')

In a quaint attic, dust swirled as I stumbled upon a peculiar puzzle. Its cryptic symbols teased my mind, beckoning me closer.  With trembling hands, I deciphered a chilling message: "He who ascends the creaky stairs shall meet their doom." Intrigued and terrified, I hesitated. But curiosity gnawed at me, compelling me to tread upward.  As I cautiously made my way, each step creaked ominously, sending shivers down my spine. At the top, I found a weathered box. Inside, a single key lay nestled.  But as I reached for it, a cold hand shot out from the shadows, its bony fingers seizing mine. A spectral laughter filled the air, echoing through the empty corridors. The attic grew darker, enveloping me in suffocating fear.
In the eerie silence of the desolate moor, a solitary figure stumbled through the dense fog. John, a renowned puzzle enthusiast, had ventured deep into the enigmatic expanse, driven by an insatiable curiosity for the macabre. As he cautiously navigated the treacherous path,

In [6]:
def speech_to_text(audio_file):
    model = whisper.load_model("base")
    data = model.transcribe(audio_file, word_timestamps=True)

    start = [data['segments'][i]['words'][j]['start'] for i in range(len(data['segments'])) for j in range(len(data['segments'][i]['words']))]
    end = [data['segments'][i]['words'][j]['end'] for i in range(len(data['segments'])) for j in range(len(data['segments'][i]['words']))]
    text = [data['segments'][i]['words'][j]['word'] for i in range(len(data['segments'])) for j in range(len(data['segments'][i]['words']))]

    return [start, end, text]

In [9]:
from moviepy.video.tools.subtitles import SubtitlesClip
from moviepy.editor import CompositeAudioClip, concatenate_audioclips

def generate_video():
    for i in range(stories):
        with open(f'stories/story_{i}.txt', 'r') as file:
            print(f'Generating video for story {i}')
            text = file.read()
        
        words = text.split(' ')

        generator = lambda txt: TextClip(txt, 
                                    fontsize=70, 
                                    color='white', 
                                    bg_color='none', 
                                    font='Arial-Bold',
                                    method = 'caption',
                                    size = (1920, 1080))

        # def clip_generator():
        #     for _ in words:
        #         yield CompositeVideoClip([ImageClip('Z_1.jpg').set_duration(0.5).set_position('center')], size=(720, 1280))

        # clips = concatenate_videoclips(clip_generator(), method="compose")

        speech = speech_to_text(f'audio/story_{i}.mp3')
        print(f'Accessing speech for story {i}')
        subs = [((speech[0][k], speech[1][k]), speech[2][k]) for k in range(len(speech[0]))]

        subtitles = SubtitlesClip(subs, generator)
        audio_clip = AudioFileClip(f'audio/story_{i}.mp3')
        video = VideoFileClip('../assets/BG.mp4')
        final_complete = CompositeVideoClip([video.set_duration(audio_clip.duration), subtitles.set_position(('center', 'center'))])
        final_complete = final_complete.set_audio(audio_clip)
        final_complete.write_videofile(f"output_{i}.mp4" , fps=24, bitrate = '1000', preset = 'fast', codec = 'h264_nvenc')
        final_complete.close()


In [11]:
generate_video()

Generating video for story 0


Accessing speech for story 0


Moviepy - Building video output_0.mp4.
MoviePy - Writing audio in output_0TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_0.mp4



Moviepy - Done !
Moviepy - video ready output_0.mp4
Generating video for story 1
Accessing speech for story 1


Moviepy - Building video output_1.mp4.
MoviePy - Writing audio in output_1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_1.mp4



t:   0%|          | 8/1763 [00:00<02:15, 12.95it/s, now=None]

KeyboardInterrupt: 